In [19]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
import time
import re
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle

#load training data
twitter_train = pd.read_excel('./StanceDataset/train.xlsx')

#load test data
twitter_test = pd.read_excel('./StanceDataset/test.xlsx')

In [2]:
twitter_train.head()

,Tweet,Target,Stance,Opinion Towards,Sentiment
0,"@tedcruz And, #HandOverTheServer she wiped cle...",Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
1,Hillary is our best choice if we truly want to...,Hillary Clinton,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
2,@TheView I think our country is ready for a fe...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
3,I just gave an unhealthy amount of my hard-ear...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
4,@PortiaABoulger Thank you for adding me to you...,Hillary Clinton,NONE,3. The tweet is not explicitly expressing opi...,pos


In [3]:
twitter_test.head()

,Tweet,Target,Stance,Opinion Towards,Sentiment
0,He who exalts himself shall be humbled; a...,Atheism,AGAINST,1. The tweet explicitly expresses opinion abo...,pos
1,RT @prayerbullets: I remove Nehushtan -previou...,Atheism,AGAINST,1. The tweet explicitly expresses opinion abo...,other
2,@Brainman365 @heidtjj @BenjaminLives I have so...,Atheism,AGAINST,1. The tweet explicitly expresses opinion abo...,pos
3,#God is utterly powerless without Human interv...,Atheism,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
4,@David_Cameron Miracles of #Multiculturalism...,Atheism,AGAINST,2. The tweet does NOT expresses opinion about ...,neg


## Train Elmo models and write outputs to pickle files for use later

# DO NOT RUN UNLESS NECESSARY - TAKES VERY LONG

In [4]:
#download elmo model
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [5]:
#function for training elmo model by passing a series of tweets
def elmo_vectors(x):
    embeddings = elmo(x.tolist(), signature="default", as_dict=True)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        # return elmo output
        return sess.run(embeddings)

start_time_train = time.time()
train_elmo = elmo_vectors(twitter_train['Tweet'])
print("Training vectors done") 
end_time_train = time.time()
print("Training vectors time:",end_time_train - start_time_train)

start_time_test = time.time()
test_elmo = elmo_vectors(twitter_test['Tweet'])
print("Test vectors done")
end_time_test = time.time()
print("Testing vectors time:",end_time_test - start_time_test)

In [29]:
#write training to pickle
pickle1 = open('elmo_train.pickle','wb')
pickle.dump(train_elmo, pickle1)
pickle1.close()

#write test to pickle
pickle2 = open('elmo_test.pickle','wb')
pickle.dump(test_elmo, pickle2)
pickle2.close()

## Code for loading Elmo vectors from pickle files

In [30]:
with open("elmo_train.pickle", "rb") as f:
    train_vectors = pickle.load(f)
    
with open("elmo_test.pickle", "rb") as f:
    test_vectors = pickle.load(f)

In [ ]:
stance_labels_train = np.array(twitter_train['Stance'].apply(lambda x: 
                                                                    2 if x == "FAVOR" else 
                                                                    (1 if x == "NONE" else 0)))

stance_labels_test = np.array(twitter_test['Stance'].apply(lambda x: 
                                                                    2 if x == "FAVOR" else 
                                                                    (1 if x == "NONE" else 0)))

In [ ]:
svm = SVC(kernel = 'linear')
svm.fit(elmo_train_new,stance_labels_train)
stance_predict = svm.predict(elmo_test_new)
f1_score(stance_labels_test, stance_predict, average = 'macro')